In [1]:
!pip install --quiet ../input/kerasapplications/keras-team-keras-applications-3b180cb
!pip install --quiet /kaggle/input/efficientnet-git

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import cv2
import re
import math

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout,\
        Flatten,GlobalAveragePooling2D,BatchNormalization, Activation

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers

import tensorflow.keras.layers as L
import tensorflow.keras.backend as K

from kaggle_datasets import KaggleDatasets

import efficientnet.keras as efn 
print(tf.__version__)

2.3.1


In [3]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

REPLICAS: 1


In [4]:
#setting
SEED = 100
DEBUG = False
WANDB = False
# TARGET_SIZE = 512
VALIDATION_SIZE = 0.2
BATCH_SIZE = 32 *REPLICAS
LEARNING_RATE = 3e-5 * REPLICAS
EPOCHS=40
MODEL_NAME = "EfficentNetB4"
N_FOLDS = 5

TTA = True
N_TTA = 4

#For BiTempered loss function
T_1 = 0.3
T_2 = 1.2
SMOOTH_FRACTION = 0.01
N_ITER = 5

HEIGHT = 500
WIDTH = 500
HEIGHT_RS = 512
WIDTH_RS = 512
CHANNELS = 3
N_CLASSES = 5

MODEL_SAVE_PATH = ""

if DEBUG:
    EPOCHS = 2
    # LEARNING_RATE = 3e-5 * REPLICAS
    # TARGET_SIZE = 512
    # BATCH_SIZE = 8*REPLICAS

In [5]:
# data augmentation @cdeotte kernel: https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96
def transform_rotation(image, height, rotation):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated
    DIM = height
    XDIM = DIM%2 #fix for size 331
    
    rotation = rotation * tf.random.uniform([1],dtype='float32')
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape(tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3])

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(rotation_matrix,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES 
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3])

def transform_shear(image, height, shear):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly sheared
    DIM = height
    XDIM = DIM%2 #fix for size 331
    
    shear = shear * tf.random.uniform([1],dtype='float32')
    shear = math.pi * shear / 180.
        
    # SHEAR MATRIX
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape(tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3])    

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(shear_matrix,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES 
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3])

# CutOut
def data_augment_cutout(image, min_mask_size=(int(HEIGHT * .1), int(HEIGHT * .1)), 
                        max_mask_size=(int(HEIGHT * .125), int(HEIGHT * .125))):
    p_cutout = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
    if p_cutout > .85: # 10~15 cut outs
        n_cutout = tf.random.uniform([], 10, 15, dtype=tf.int32)
        image = random_cutout(image, HEIGHT, WIDTH, 
                              min_mask_size=min_mask_size, max_mask_size=max_mask_size, k=n_cutout)
    elif p_cutout > .6: # 5~10 cut outs
        n_cutout = tf.random.uniform([], 5, 10, dtype=tf.int32)
        image = random_cutout(image, HEIGHT, WIDTH, 
                              min_mask_size=min_mask_size, max_mask_size=max_mask_size, k=n_cutout)
    elif p_cutout > .25: # 2~5 cut outs
        n_cutout = tf.random.uniform([], 2, 5, dtype=tf.int32)
        image = random_cutout(image, HEIGHT, WIDTH, 
                              min_mask_size=min_mask_size, max_mask_size=max_mask_size, k=n_cutout)
    else: # 1 cut out
        image = random_cutout(image, HEIGHT, WIDTH, 
                              min_mask_size=min_mask_size, max_mask_size=max_mask_size, k=1)

    return image

def random_cutout(image, height, width, channels=3, min_mask_size=(10, 10), max_mask_size=(80, 80), k=1):
    assert height > min_mask_size[0]
    assert width > min_mask_size[1]
    assert height > max_mask_size[0]
    assert width > max_mask_size[1]

    for i in range(k):
      mask_height = tf.random.uniform(shape=[], minval=min_mask_size[0], maxval=max_mask_size[0], dtype=tf.int32)
      mask_width = tf.random.uniform(shape=[], minval=min_mask_size[1], maxval=max_mask_size[1], dtype=tf.int32)

      pad_h = height - mask_height
      pad_top = tf.random.uniform(shape=[], minval=0, maxval=pad_h, dtype=tf.int32)
      pad_bottom = pad_h - pad_top

      pad_w = width - mask_width
      pad_left = tf.random.uniform(shape=[], minval=0, maxval=pad_w, dtype=tf.int32)
      pad_right = pad_w - pad_left

      cutout_area = tf.zeros(shape=[mask_height, mask_width, channels], dtype=tf.uint8)

      cutout_mask = tf.pad([cutout_area], [[0,0],[pad_top, pad_bottom], [pad_left, pad_right], [0,0]], constant_values=1)
      cutout_mask = tf.squeeze(cutout_mask, axis=0)
      image = tf.multiply(tf.cast(image, tf.float32), tf.cast(cutout_mask, tf.float32))

    return image

In [6]:
def data_augment(image, label):
    p_rotation = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_shear = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_cutout = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    flip_left = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    flip_right = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
#     # Shear
#     if p_shear > .2:
#         if p_shear > .6:
#             image = transform_shear(image, HEIGHT, shear=20.)
#         else:
#             image = transform_shear(image, HEIGHT, shear=-20.)
            
    # Rotation
    if p_rotation > .2:
        if p_rotation > .6:
            image = transform_rotation(image, HEIGHT, rotation=10.)
        else:
            image = transform_rotation(image, HEIGHT, rotation=-10.)
            
    # Flips
    if flip_left>0.5:
        image = tf.image.random_flip_left_right(image)
    if flip_right>0.5 :
        image = tf.image.random_flip_up_down(image)
#     if p_spatial > .75:
#         image = tf.image.transpose(image)
        
#     # Rotates
#     if p_rotate > .75:
#         image = tf.image.rot90(image, k=3) # rotate 270º
#     elif p_rotate > .5:
#         image = tf.image.rot90(image, k=2) # rotate 180º
#     elif p_rotate > .25:
#         image = tf.image.rot90(image, k=1) # rotate 90º
        
#     # Pixel-level transforms
#     if p_pixel_1 >= .4:
#         image = tf.image.random_saturation(image, lower=.7, upper=1.3)
#     if p_pixel_2 >= .4:
#         image = tf.image.random_contrast(image, lower=.8, upper=1.2)
#     if p_pixel_3 >= .4:
#         image = tf.image.random_brightness(image, max_delta=.1)
        
#     # Crops
#     if p_crop > .6:
#         if p_crop > .9:
#             image = tf.image.central_crop(image, central_fraction=.5)
#         elif p_crop > .8:
#             image = tf.image.central_crop(image, central_fraction=.6)
#         elif p_crop > .7:
#             image = tf.image.central_crop(image, central_fraction=.7)
#         else:
#             image = tf.image.central_crop(image, central_fraction=.8)
#     elif p_crop > .3:
#         crop_size = tf.random.uniform([], int(HEIGHT*.6), HEIGHT, dtype=tf.int32)
#         image = tf.image.random_crop(image, size=[crop_size, crop_size, CHANNELS])
            
    image = tf.image.resize(image, size=[HEIGHT, WIDTH])

#     if p_cutout > .5:
#         image = data_augment_cutout(image)
        
    return image, label

In [7]:
def center_crop(image):
#     image = tf.reshape(image, [600, 800, CHANNELS]) # Original shape
    image = tf.image.resize(image, [600, 800])
    
    h, w = image.shape[0], image.shape[1]
    if h > w:
        image = tf.image.crop_to_bounding_box(image, (h - w) // 2, 0, w, w)
    else:
        image = tf.image.crop_to_bounding_box(image, 0, (w - h) // 2, h, h)
        
#     image = tf.image.resize(image, [HEIGHT, WIDTH]) # Expected shape
    return image

# Datasets utility functions
def decode_image(image_data):
    """
        1. Decode a JPEG-encoded image to a uint8 tensor.
        2. Cast tensor to float and normalizes (range between 0 and 1).
        3. Resize and reshape images to the expected size.
    """
    image = tf.image.decode_jpeg(image_data, channels=3)
    
    image = tf.cast(image, tf.float32)/255.0
    
#     image = center_crop(image)
    
    
   
                      
    image = tf.image.resize(image, [HEIGHT, WIDTH])
    image = tf.reshape(image, [HEIGHT, WIDTH, 3])
    return image

def read_tfrecord(example, labeled=True):
    """
        1. Parse data based on the 'TFREC_FORMAT' map.
        2. Decode image.
        3. If 'labeled' returns (image, label) if not (image, name).
    """
    if labeled:
        TFREC_FORMAT = {
            'image': tf.io.FixedLenFeature([], tf.string), 
            'target': tf.io.FixedLenFeature([], tf.int64), 
        }
    else:
        TFREC_FORMAT = {
            'image': tf.io.FixedLenFeature([], tf.string), 
            'image_name': tf.io.FixedLenFeature([], tf.string), 
        }
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    image = decode_image(example['image'])
   
    if labeled:
        label_or_name = tf.cast(example['target'], tf.int32)
    else:
        label_or_name =  example['image_name']
    return image, label_or_name

def load_dataset(filenames, labeled=True, ordered=False):
    """
        Create a Tensorflow dataset from TFRecords.
    """
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(lambda x: read_tfrecord(x, labeled=labeled), num_parallel_calls=AUTO)
    return dataset

def get_dataset(FILENAMES, labeled=True, ordered=False, repeated=False, augment=False):
    """
        Return a Tensorflow dataset ready for training or inference.
    """
    dataset = load_dataset(FILENAMES, labeled=labeled, ordered=ordered)
    if augment:
        dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    if repeated:
        dataset = dataset.repeat()
    if not ordered:
        dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

In [8]:
# import module we'll need to import our custom module
from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/bitempered-logistic-loss-tensorflow-v2/bi_tempered_loss.py", dst = "../working/loss.py")

# import all our functions
from loss import bi_tempered_logistic_loss

In [9]:
with strategy.scope():
  class BiTemperedLogisticLoss(tf.keras.losses.Loss):
    def __init__(self, t1, t2, lbl_smth, n_iter):
      super(BiTemperedLogisticLoss, self).__init__()
      self.t1 = t1
      self.t2 = t2
      self.lbl_smth = lbl_smth
      self.n_iter = n_iter

    def call(self, y_true, y_pred):
      return bi_tempered_logistic_loss(y_pred, y_true, self.t1, self.t2, self.lbl_smth, self.n_iter)

In [10]:
# GCS_PATH= KaggleDatasets().get_gcs_path('cassava-leaf-disease-classification')+'/test_tfrecords' 
files_path = '../input/cassava-leaf-disease-classification/test_images'

TEST_FILENAMES = tf.io.gfile.glob('../input/cassava-leaf-disease-classification/test_tfrecords/*')
# print(TEST_FILENAMES)
model_base_path = '../input/cas-inference/'

model_path_list = os.listdir(model_base_path)
# print(model_path_list)

# model_path_list = [x for x in model_path_list if x.startswith('EfficentNet4_best_btl_') ]

model_path_list = [ 
    'seresnext50_fold0_best.h5',
    'seresnext50_fold1_best.h5',
    'seresnext50_fold2_best.h5',
    'seresnext50_fold3_best.h5',
#     'seresnext50_fold0_best.h5',
#     'seresnext50_fold1_best.h5',
#     'seresnext50_fold2_best.h5',
#     'seresnext50_fold3_last.h5',
]

test_preds = np.zeros((len(os.listdir(files_path)), N_CLASSES))

for model_path in model_path_list:
    print("Model: ", model_path)
    tf.keras.backend.clear_session()
#     print(os.path.join(model_base_path,model_path))
    model = tf.keras.models.load_model(os.path.join(model_base_path,model_path) )
#                                       custom_objects={'loss': BiTemperedLogisticLoss(t1=T_1, t2=T_2, lbl_smth=SMOOTH_FRACTION, n_iter=N_ITER)},
#                                        compile=False)
    
    if TTA:
        
        for step in range(N_TTA):
            print(f"TTA step: {step+1}/{N_TTA}")
            test_ds = get_dataset(TEST_FILENAMES, labeled=False, ordered=True, augment = True)
            x_test = test_ds.map(lambda image, image_name: image)
            test_preds += model.predict(x_test)/len(model_path_list)
        
    
    else:
        test_ds = get_dataset(TEST_FILENAMES, labeled=False, ordered=True)
        x_test = test_ds.map(lambda image, image_name: image)
        test_preds += model.predict(x_test)/len(model_path_list)

Model:  seresnext50_fold0_best.h5
TTA step: 1/4
TTA step: 2/4
TTA step: 3/4
TTA step: 4/4
Model:  seresnext50_fold1_best.h5
TTA step: 1/4
TTA step: 2/4
TTA step: 3/4
TTA step: 4/4
Model:  seresnext50_fold2_best.h5
TTA step: 1/4
TTA step: 2/4
TTA step: 3/4
TTA step: 4/4
Model:  seresnext50_fold3_best.h5
TTA step: 1/4
TTA step: 2/4
TTA step: 3/4
TTA step: 4/4


In [11]:
test_preds = np.argmax(test_preds, axis=-1)
image_names = [img_name.numpy().decode('utf-8') for img, img_name in iter(test_ds.unbatch())]

submission = pd.DataFrame({'image_id': image_names, 'label': test_preds})
submission.to_csv('submission.csv', index=False)
display(submission.head())

,image_id,label
0,2216849948.jpg,4


In [12]:
!pip install efficientnet

In [13]:
# from matplotlib import image as img
# from keras import models
# import pandas as pd
# import numpy as np
# import os

In [14]:
# tf.keras.models.load_model('../input/cassava-inference/EfficentNetB4_best_fold_0_.h5',
#                                       custom_objects={'loss': BiTemperedLogisticLoss(t1=T_1, t2=T_2, lbl_smth=SMOOTH_FRACTION, n_iter=N_ITER)},
#                                        compile=False)

In [15]:
# model1 = models.load_model('../input/cassava-inference/EfficentNetB4_best_fold_0_.h5')